```
https://stackoverflow.com/questions/76603587/non-linear-data-fitting-for-kinetic-data

```
solve {x = 1/(1+A+sqrt(s*y/x)), y = 1/(1+A+B +sqrt(s*x/y))} for x and y
```

https://www.wolframalpha.com/input?i=solve+%7Bx+%3D+1%2F%281%2BA%2Bsqrt%28s*y%2Fx%29%29%2C+y+%3D+1%2F%281%2BA%2BB+%2Bsqrt%28s*x%2Fy%29%29%7D+for+x+and+y

```
jacobian {1/(1+B+sqrt(s*y/x)), 1/(1+A+C +sqrt(s*x/y))} wrt x and y
```

https://www.wolframalpha.com/input?i=jacobian+%7B1%2F%281%2BB%2Bsqrt%28s*y%2Fx%29%29%2C+1%2F%281%2BA%2BC+%2Bsqrt%28s*x%2Fy%29%29%7D+wrt+x+and+y

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize 
import logging
import functools

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [3]:
class Kinetic:
    
    """Langmuir-Hinshelwood Two-Site Kinetic Model Solver"""
    
    R = 8.314              # J/mol.K
    Ea = 37000             # J/mol
    T = 473.15             # K
    theta_ = [0.9, 0.9]    # 1
    
    @staticmethod
    def k3(p, k1, k2, k4, k5, k0):
        """Arrhenius estimation for k3"""
        return k0*np.exp(-Kinetic.Ea/(Kinetic.R*Kinetic.T))

    @staticmethod
    def s(p, k1, k2, k4, k5, k0):
        """Partial kinetic rate"""
        pA, pB, pC = p
        k3 = Kinetic.k3(p, k1, k2, k4, k5, k0)
        return (k1*k2*k3/k4)*pA*pB
    
    @staticmethod
    def equation(x, y, C, s):
        return 1/(1 + C + np.sqrt(s*y/x)) - x
    
    @staticmethod
    def isopleth(x, y, s):
        return (x + np.sqrt(s*x*y) - 1)/x
    
    @staticmethod
    def system(theta, p, k1, k2, k4, k5, k0):
        """Isothermal coverages system"""
        pA, pB, pC = p
        t0, t1 = theta
        s = Kinetic.s(p, k1, k2, k4, k5, k0)
        C0 = k2*pB
        C1 = k1*pA + pC/k5
        return np.array([
            Kinetic.equation(t0, t1, C0, s),
            Kinetic.equation(t1, t0, C1, s),
        ])
    
    @staticmethod
    def _theta(p, k1, k2, k4, k5, k0):
        """Single Isothermal coverages"""
        return optimize.fsolve(
            Kinetic.system,
            Kinetic.theta_,
            args=(p, k1, k2, k4, k5, k0),
            full_output=False
        )
    
    @staticmethod
    def theta(p, k1, k2, k4, k5, k0):
        """Isothermal coverages"""
        return np.apply_along_axis(Kinetic._theta, 0, p, k1, k2, k4, k5, k0)
    
    @staticmethod
    def r(p, k1, k2, k4, k5, k0):
        """Global kinetic rate"""
        pA, pB, pC = p
        k3 = Kinetic.k3(p, k1, k2, k4, k5, k0)
        t0, t1 = Kinetic.theta(p, k1, k2, k4, k5, k0)
        return (k1*k2*k3)*pA*pB*t0*t1
    
    @staticmethod
    def solve(p, r, *k):
        """Global kinetic constants adjustment"""
        return optimize.curve_fit(
            Kinetic.r, p.T, r, p0=k,
            bounds=((0, 0, 0, 0, 0), (np.inf, np.inf, np.inf, np.inf, np.inf)),
            method="trf",
            gtol=1e-10,
            full_output=True
        )
    
    @staticmethod
    def eval(p, k1, k2, k4, k5, k0):
        return np.vectorize(Kinetic.r, excluded=["p"])(p, k1, k2, k4, k5, k0)
    
    @staticmethod
    def mse(k1, k2, k4, k5, k0, p, r):
        pass
    
    @staticmethod
    def abaque(s, C=None):
        
        lin = np.linspace(0.0001, 1.1, 200)
        X, Y = np.meshgrid(lin, lin)
        if C is None:
            C = np.arange(0, 26, 1)
        
        Cx = Kinetic.isopleth(X, Y, s)
        Cy = Kinetic.isopleth(Y, X, s)
      
        fig, axe = plt.subplots()
        xlabels = axe.contour(X, Y, Cx, C, cmap="jet")
        axe.clabel(xlabels, xlabels.levels, inline=True, fontsize=7)
        ylabels = axe.contour(X, Y, Cy, C, cmap="jet")
        axe.clabel(ylabels, ylabels.levels, inline=True, fontsize=7)
        axe.set_title("Isothemal Coverages System\nConstant Isopleths ($s={}$)".format(s))
        axe.set_xlabel(r"Partial Coverage, $\theta_0$ [-]")
        axe.set_ylabel(r"Partial Coverage, $\theta_1$ [-]")
        axe.set_aspect("equal")
        axe.set_xlim([0, 1.1])
        axe.set_ylim([0, 1.1])
        axe.grid()
        
        return axe

In [4]:
df = pd.read_excel("LangmuirData.xlsx")
df

,pa,pb,pc,T,rate
0,0.10,0.11,0.0114,473.15,0.000001
1,0.13,0.11,0.0121,473.15,0.000002
2,0.20,0.11,0.0170,473.15,0.000004
3,0.25,0.11,0.0172,473.15,0.000004
4,0.30,0.11,0.0252,473.15,0.000005
5,0.35,0.11,0.0261,473.15,0.000006
6,0.10,0.16,0.0123,473.15,0.000003
7,0.10,0.20,0.0172,473.15,0.000004
8,0.10,0.22,0.0181,473.15,0.000003
9,0.10,0.33,0.0221,473.15,0.000005


In [5]:
p = df.filter(regex="p").values
r = df["rate"].values

In [6]:
#Kinetic.solve(p, r, 5, 1e5, 50, 10, 100)

In [7]:
#Kinetic.solve(p, r, 1, 1, 1, 1, 1)

In [8]:
#Kinetic.solve(p, r, 1e2, 1e2, 1e2, 1e2, 1e2)

In [9]:
plin = np.linspace(0.1, 0.3, 5)
PA, PB, PC = np.meshgrid(plin, plin, plin)

In [10]:
P = np.vstack([
    PA.flatten(), PB.flatten(), PC.flatten()
]).T

In [11]:
R = Kinetic.r(P.T, 5, 1e5, 50, 10, 100)

In [12]:
R.shape

(125,)

In [13]:
#Kinetic.solve(P, R, 1e2, 1e2, 1e2, 1e2, 1e2)

In [14]:
#Kinetic.solve(P, R, 1e4, 1e4, 1e4, 1e4, 1e4)

In [15]:
#Kinetic.solve(P, R, 1, 1, 1, 1, 1)

In [42]:
k1, k2, k4, k5, k0 = 5, 1e5, 50, 10, 100
N = 10
k1l = np.logspace(-2, 2, N, base=10)
k2l = np.logspace(3, 7, N, base=10)
k4l = np.logspace(0, 4, N, base=10)
k5l = np.logspace(-1, 3, N, base=10)
k0l = np.logspace(0, 4, N, base=10)

In [43]:
#K1, K2, K4, K5, K0 = np.meshgrid(k1l, k2l, k4l, k5l, k0l)

In [44]:
partial = functools.partial(Kinetic.r, P.T)
vectorized = np.vectorize(partial, otypes=[np.ndarray])

In [45]:
K1, K2 = np.meshgrid(k1l, k2l)

In [46]:
Z = vectorized(K1, K2, k4, k5, k0)

In [56]:
vectorized(k1, k2, k4, k5, k0)

array([0.0026835085593803973, 0.0026747168183653774,
       0.0026659826015079323, 0.0026573053455674067,
       0.0026486844946365853, 0.003446700347732616, 0.0034370172204688013,
       0.0034273884609138574, 0.0034178136119904713,
       0.0034082922217339266, 0.0040186342729303165,
       0.0040087517087486245, 0.003998917740412106, 0.003989132009956464,
       0.003979394162926996, 0.0044632655989774225, 0.004453505851283946,
       0.004443788794413017, 0.004434114148539868, 0.004424481636281743,
       0.004818866832019479, 0.004809380492174581, 0.004799931520183615,
       0.0047905196954146955, 0.004781144798970756, 0.00268359734841803,
       0.0026748053175821482, 0.0026660708127949017,
       0.0026573932707971613, 0.002648772135663475, 0.0034468142762337024,
       0.0034371308302372835, 0.003427501753733712, 0.003417926589630715,
       0.0034084048859487855, 0.004018767017613897, 0.00400888412846235,
       0.003999049836748544, 0.003989263784496489, 0.003979525617239897

In [ ]:
fig, axe = plt.subplots()
axe.contour(K1, K2, Z)